In [16]:
import pandas as pd
import numpy as np

# Preprocess covariates

In [19]:
oil_raw = pd.read_csv('data/oil.csv', parse_dates=["date"], index_col="date")
oil = pd.DataFrame()
oil["date"] = np.arange(oil_raw.index.min(), oil_raw.index.max() + np.timedelta64(7, "D"), np.timedelta64(1, "D"), dtype="datetime64[D]")
oil = oil.join(oil_raw, on="date", how="left").bfill().ffill()
oil.date = oil.date.astype("datetime64[ns]")
oil.to_csv("gen_data/oil.csv", index=False)

In [18]:
stores = pd.read_csv("data/stores.csv")
holidays = pd.read_csv("data/holidays_events.csv", parse_dates=["date"])

eq = holidays.description.str.startswith("Terremoto Manabi")
eq_days = holidays[eq]
holidays = holidays[~eq]

holidays
stores = stores[["store_nbr", "city", "state"]]

r_holidays = stores.merge(holidays[holidays.locale == "Regional"], left_on="state", right_on="locale_name", how="inner")[["store_nbr", "date", "locale"]]
l_holidays = stores.merge(holidays[holidays.locale == "Local"], left_on="city", right_on="locale_name", how="inner")[["store_nbr", "date", "locale"]]
n_holidays = stores.merge(holidays[holidays.locale == "National"], how="cross")[["store_nbr", "date", "locale"]]
eq_days["locale"] = "Earthquake"
eq_holidays = stores.merge(eq_days, how="cross")[["store_nbr", "date", "locale"]]

holidays = pd.concat([r_holidays, l_holidays, n_holidays, eq_holidays])
holidays = holidays.drop_duplicates()
holidays = pd.get_dummies(holidays, columns=["locale"])
holidays = holidays.groupby(["store_nbr", "date"]).sum().reset_index()
holidays = holidays.sort_values(["date"])

holidays.to_csv("gen_data/holidays.csv", index=False)

# Create the Uber Dataset

In [5]:
stores = pd.read_csv("data/stores.csv")
stores = stores[["store_nbr", "type", "cluster"]].rename(columns={"type": "store_type", "cluster": "store_cluster"})
stores = pd.get_dummies(stores, columns=["store_type", "store_cluster"])
dummies = list(set(stores.columns) - {"store_nbr"})
stores[dummies] = stores[dummies].astype("uint8")
stores.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   store_nbr         54 non-null     int64
 1   store_type_A      54 non-null     uint8
 2   store_type_B      54 non-null     uint8
 3   store_type_C      54 non-null     uint8
 4   store_type_D      54 non-null     uint8
 5   store_type_E      54 non-null     uint8
 6   store_cluster_1   54 non-null     uint8
 7   store_cluster_2   54 non-null     uint8
 8   store_cluster_3   54 non-null     uint8
 9   store_cluster_4   54 non-null     uint8
 10  store_cluster_5   54 non-null     uint8
 11  store_cluster_6   54 non-null     uint8
 12  store_cluster_7   54 non-null     uint8
 13  store_cluster_8   54 non-null     uint8
 14  store_cluster_9   54 non-null     uint8
 15  store_cluster_10  54 non-null     uint8
 16  store_cluster_11  54 non-null     uint8
 17  store_cluster_12  54 non-null     uin

In [6]:
oil = pd.read_csv("gen_data/oil.csv", parse_dates=["date"])
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703 entries, 0 to 1702
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1703 non-null   datetime64[ns]
 1   dcoilwtico  1703 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 26.7 KB


In [7]:
holidays = pd.read_csv("gen_data/holidays.csv", parse_dates=["date"])
locale = [c for c in holidays.columns if c.startswith("locale")]
holidays[locale] = holidays[locale].fillna(0).astype("uint8")
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601 entries, 0 to 9600
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   store_nbr          9601 non-null   int64         
 1   date               9601 non-null   datetime64[ns]
 2   locale_Earthquake  9601 non-null   uint8         
 3   locale_Local       9601 non-null   uint8         
 4   locale_National    9601 non-null   uint8         
 5   locale_Regional    9601 non-null   uint8         
dtypes: datetime64[ns](1), int64(1), uint8(4)
memory usage: 187.6 KB


In [10]:
def aug(df):
    df = df.merge(stores, left_on="store_nbr", right_on="store_nbr", how="inner")
    df = df.merge(oil, left_on="date", right_on="date", how="inner")
    df = df.merge(holidays, left_on=["date", "store_nbr"], right_on=["date", "store_nbr"], how="left")
    df[locale] = df[locale].fillna(0).astype("uint8")

    df["month"] = df.date.dt.month.astype("uint8")
    df["year"] = df.date.dt.year.astype("uint16")
    df["doy"] = df.date.dt.dayofyear.astype("uint16")
    df["dow"] = df.date.dt.dayofweek.astype("uint8")
    df = df.drop(columns=["date"])

    df = pd.get_dummies(df, columns=["family", "dow", "month", "store_nbr"])
    d = [c for c in df.columns if c.startswith("dow_") or c.startswith("month_") or c.startswith("family_") or c.startswith("store_nbr_")]
    df[d] = df[d].astype("uint8")
    return df

In [11]:
train = pd.read_csv('data/train.csv', parse_dates=["date"])
new_train = aug(train)
new_train.to_csv("gen_data/train.csv", index=False)

In [13]:
test = pd.read_csv('data/test.csv', parse_dates=["date"])
new_test = aug(test)
for col in new_train.columns:
    if col not in new_test.columns:
        new_test[col] = 0
new_test = new_test[new_train.columns]
new_test = new_test.drop(columns=["sales"])
new_test.to_csv("gen_data/test.csv", index=False)

/var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/ipykernel_54057/1145219075.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_test[col] = 0
/var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/ipykernel_54057/1145219075.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_test[col] = 0
/var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/ipykernel_54057/1145219075.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join